<H1>Probabilistic Deep Learning</H1>
<H2>Generative Models</H2>


$\text{Generative models provide a mechanism to generate samples from a given distribution.}$<br>



<H2>Deep Probabilistic Generative Models</H2>
<H2>Latent Variables</H2>
<H2>Diffusion Models</H2>


<H2>Variational Inference</H2>
<H2>Variational Auto Encoder</H2>


$x_{0}\overset{forward}{\underset{reverse}\rightleftarrows} x_{1:T}$<br>
$\text{where}$<br>
$x_{1:T}= x_{0}\rightarrow x_{1}\rightarrow\dots\rightarrow x_{T}$<br>
$\text{and}$<br>
$x_{t+1}= x_{t-1}$<br> 


Each latent variable $x_{t}\in x_{1:t}$ consists of the sum of noise and the value of $x_{t-1}$.<br>
The joint distribution $q(x_{1:T}|x_{0})$ of latent variables $x_{1:T}$ given the original manifest variable $x_{0}$ is memoryless.<br>
$q(x_{t:T}|x_{0})=\prod_{t=1}^{T}{q(x_{t}|x_{t-1})}\quad\quad\quad\quad\text{(Markov Property)}$<br>

Let $q(x_{t}|x_{t_1})\sim\mathcal{N}(x_{t};\sqrt{1-\beta}x_{t-1},\beta_{t}I)\quad\quad\quad\quad\quad\quad\quad\text{(where}\beta_{t}\text{is the variance at time t)}$<br>